In [18]:
import numpy as np
from scipy.stats import qmc
np.random.seed(1234)
# Define the hyperparameter grid
args = {
    'n_embed': [128, 256, 512],
    'e_embed': [128, 256, 512],
    'aggs': ['sum mean min max var std', 'sum min max', 'sum mean std', 
             'mean min max std', 'mean std', 'sum mean min max std'],
    'scalers': ['identity amplification attenuation linear inverse_linear',
                'identity amplification attenuation',
                'attenuation linear inverse_linear',
                'amplification attenuation',
                'identity linear inverse_linear',
                'amplification attenuation linear inverse_linear',
                'identity amplification attenuation linear',
                'identity linear'],
    'n_pna': [2, 3, 4, 5],
    'hidden': [64, 128, 256, 512],
    'n_mlp_layer': [2, 3, 4, 5],
    'af': ['relu', 'elu', 'silu', 'tanh'],
    'batch_size': [32, 64, 128],
    'lr': [0.001, 0.0001, 0.00001],
    'weight_decay': [1e-4, 1e-5, 1e-6],
    'es_episodes': [20, 50, 70],
    'es_improvement': [0, 1e-3, 1e-4],
    'epochs': [200, 400, 600],
}

# Number of samples
n_samples = 50  

# Convert categorical hyperparameters to numerical indices
param_keys = list(args.keys())
param_sizes = [len(args[key]) for key in param_keys]

# Generate LHS samples
sampler = qmc.LatinHypercube(d=len(param_keys), seed=1234)
lhs_samples = sampler.random(n=n_samples)

# Scale samples to match parameter indices
config = {}
for i in range(n_samples):
    model_config = 'python main.py '
    config[i] = []
    
    for j, key in enumerate(param_keys):
        index = int(lhs_samples[i, j] * param_sizes[j])  # Scale to discrete indices
        elem = args[key][index]  
        model_config += f'--{key} {elem} '
        config[i].append(elem)
    
    print(model_config)

python main.py --n_embed 256 --e_embed 256 --aggs mean min max std --scalers amplification attenuation --n_pna 5 --hidden 64 --n_mlp_layer 4 --af silu --batch_size 32 --lr 0.0001 --weight_decay 1e-05 --es_episodes 70 --es_improvement 0 --epochs 200 
python main.py --n_embed 128 --e_embed 512 --aggs sum min max --scalers identity amplification attenuation --n_pna 5 --hidden 128 --n_mlp_layer 5 --af elu --batch_size 64 --lr 1e-05 --weight_decay 1e-06 --es_episodes 50 --es_improvement 0.001 --epochs 600 
python main.py --n_embed 256 --e_embed 256 --aggs mean min max std --scalers identity linear inverse_linear --n_pna 4 --hidden 256 --n_mlp_layer 3 --af relu --batch_size 64 --lr 0.0001 --weight_decay 1e-06 --es_episodes 20 --es_improvement 0.0001 --epochs 400 
python main.py --n_embed 512 --e_embed 256 --aggs sum min max --scalers identity linear inverse_linear --n_pna 2 --hidden 256 --n_mlp_layer 2 --af relu --batch_size 128 --lr 1e-05 --weight_decay 0.0001 --es_episodes 70 --es_improvem

In [19]:
import pandas as pd

df = pd.DataFrame(config)

In [20]:
df = df.T

In [21]:
df.to_csv('config.csv', index=False)